In [1]:
## Parameters specific to where your folders are and your data
parameter_file = '../parameters/params.yaml'
import yaml
import sys
import glob
with open(parameter_file,'r') as f:
    doc = yaml.load(f)

#p = dic2obj(**doc)

data_folder = doc['data_folder']
tissues = doc['tissues'].split(',')
sys.path.append(doc['pipeline_path'])
ref_fa = doc['ref_fa']
annotation= doc['annotation']
#annotation_gtf = doc['annotation_gtf']


import os

import sys
import pandas as pd
import matplotlib
import seaborn as sns
import pickle
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from itertools import product
import glob
import re
from matplotlib_venn import venn2, venn3
from matplotlib import rcParams
import inspect
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
sys.setrecursionlimit(3000)
%load_ext autoreload
%autoreload 2
rcParams['figure.figsize'] = 8, 6
import tqdm
from os.path import basename, join
from Homer import *

print('Number of tissues: ',len(tissues))

import helper

from mplh.color_utils import get_colors
from mplh.fig_utils import legend_from_color

/home/isshamie/software/anaconda2/envs/TSS/lib/python3.7/site-packages/ipykernel_launcher.py:7: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  import sys


Number of tissues:  13


In [2]:
from matplotlib_venn import venn2

from tss.data import generate_genome
from tss.data.cumsum import *

params {'EMAIL': 'isshamie@ucsd.edu', 'parameters': 'parameters/params.yaml'}
/data2/resources/software/homer/v4.11_10242019/bin//


### Input

In [3]:
os.chdir(doc["results"])

fig1_peaks = "figure1_summary"

mrna_peak = pd.read_csv(join("genome", "picr", "gcf","mRNA.peak"),sep='\t')

all_genes_f = join("genome", "picr", "gcf","all_genes.tsv")
genes_silenced = pd.read_csv(join(fig1_peaks, "genes_silenced.csv"), header=None)

mrna_peak["Dbxref"]



0        GeneID:100754303,Genbank:XM_027386529.1
1        GeneID:100754303,Genbank:XM_027386530.1
2        GeneID:100754303,Genbank:XM_027386528.1
3        GeneID:100752216,Genbank:XM_027386531.1
4        GeneID:100752216,Genbank:XM_027386532.1
                          ...                   
46746    GeneID:103158952,Genbank:XM_027434593.1
46747    GeneID:113839149,Genbank:XM_027434595.1
46748    GeneID:113839149,Genbank:XM_027434594.1
46749    GeneID:113839149,Genbank:XM_027434596.1
46750    GeneID:113839150,Genbank:XM_027434597.1
Name: Dbxref, Length: 46751, dtype: object

## Outdir :

In [4]:
outdir = "gene_ids"
if not os.path.exists(outdir):
    os.mkdir(outdir)

## Need to split the Dbxref key

In [5]:
mrna_peak["GeneID"] = ""
for ind, val in mrna_peak.iterrows(): 
    curr = val["Dbxref"].split(",")
    for c in curr:
        curr2 = c.split(":")
        if curr2[0] == "GeneID":
            #print(curr2)
            mrna_peak.loc[ind,"GeneID"] = curr2[1]
            
mrna_peak

,transcript_id,Chr,Start,End,Strand,actual_start,ID,Parent,Dbxref,Name,...,gene,model_evidence,product,Note,exception,end_range,partial,start_range,inference,GeneID
0,XM_027386529.1,NW_020822366.1,722307,722309,-,722309,rna-XM_027386529.1,gene-Bmp4,"GeneID:100754303,Genbank:XM_027386529.1",XM_027386529.1,...,Bmp4,Supporting evidence includes similarity to: 29...,bone morphogenetic protein 4%2C transcript var...,NaN,NaN,NaN,NaN,NaN,NaN,100754303
1,XM_027386530.1,NW_020822366.1,722307,722309,-,722309,rna-XM_027386530.1,gene-Bmp4,"GeneID:100754303,Genbank:XM_027386530.1",XM_027386530.1,...,Bmp4,Supporting evidence includes similarity to: 29...,bone morphogenetic protein 4%2C transcript var...,NaN,NaN,NaN,NaN,NaN,NaN,100754303
2,XM_027386528.1,NW_020822366.1,724609,724611,-,724611,rna-XM_027386528.1,gene-Bmp4,"GeneID:100754303,Genbank:XM_027386528.1",XM_027386528.1,...,Bmp4,Supporting evidence includes similarity to: 29...,bone morphogenetic protein 4%2C transcript var...,NaN,NaN,NaN,NaN,NaN,NaN,100754303
3,XM_027386531.1,NW_020822366.1,1085434,1085436,+,1085434,rna-XM_027386531.1,gene-Cdkn3,"GeneID:100752216,Genbank:XM_027386531.1",XM_027386531.1,...,Cdkn3,Supporting evidence includes similarity to: 5 ...,cyclin dependent kinase inhibitor 3%2C transcr...,NaN,NaN,NaN,NaN,NaN,NaN,100752216
4,XM_027386532.1,NW_020822366.1,1085434,1085436,+,1085434,rna-XM_027386532.1,gene-Cdkn3,"GeneID:100752216,Genbank:XM_027386532.1",XM_027386532.1,...,Cdkn3,Supporting evidence includes similarity to: 2 ...,cyclin dependent kinase inhibitor 3%2C transcr...,NaN,NaN,NaN,NaN,NaN,NaN,100752216
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46746,XM_027434593.1,NW_020824186.1,29431,29433,+,29431,rna-XM_027434593.1,gene-LOC103158952,"GeneID:103158952,Genbank:XM_027434593.1",XM_027434593.1,...,LOC103158952,Supporting evidence includes similarity to: 2 ...,PRAME family member 12-like%2C transcript vari...,NaN,NaN,NaN,NaN,NaN,NaN,103158952
46747,XM_027434595.1,NW_020824188.1,2146,2148,+,2146,rna-XM_027434595.1,gene-LOC113839149,"GeneID:113839149,Genbank:XM_027434595.1",XM_027434595.1,...,LOC113839149,Supporting evidence includes similarity to: 25...,zinc finger protein 431-like%2C transcript var...,NaN,NaN,NaN,NaN,NaN,NaN,113839149
46748,XM_027434594.1,NW_020824188.1,2146,2148,+,2146,rna-XM_027434594.1,gene-LOC113839149,"GeneID:113839149,Genbank:XM_027434594.1",XM_027434594.1,...,LOC113839149,Supporting evidence includes similarity to: 30...,zinc finger protein 431-like%2C transcript var...,NaN,NaN,NaN,NaN,NaN,NaN,113839149
46749,XM_027434596.1,NW_020824188.1,2146,2148,+,2146,rna-XM_027434596.1,gene-LOC113839149,"GeneID:113839149,Genbank:XM_027434596.1",XM_027434596.1,...,LOC113839149,Supporting evidence includes similarity to: 27...,zinc finger protein 431-like%2C transcript var...,NaN,NaN,NaN,NaN,NaN,NaN,113839149


In [6]:
mrna_peak.to_csv(join(outdir, "mrna_peak_geneID.csv"), index=False)

### Convert genes_silenced to gene_id

In [7]:
genes = mrna_peak[["gene","GeneID"]]
genes = genes[~(genes.duplicated())]
assert(len(genes["GeneID"].unique())==genes.shape[0])
print(len(genes["gene"].unique()),genes.shape[0])
genes

21387 21394


,gene,GeneID
0,Bmp4,100754303
3,Cdkn3,100752216
6,Cnih1,103160710
7,Gmfb,100754009
8,Cgrrf1,100751644
...,...,...
46743,LOC113839146,113839146
46744,LOC113839147,113839147
46746,LOC103158952,103158952
46747,LOC113839149,113839149


### 7 overall genes have duplicate GeneIDs

In [8]:
genes_silenced["geneID set"] = genes_silenced[0].apply(lambda x: set(mrna_peak[mrna_peak["gene"]==x]["GeneID"].values))
genes_silenced

,0,geneID set
0,Il20ra,{100751800}
1,LOC100756024,{100756024}
2,LOC100763587,{100763587}
3,LOC100754253,{100754253}
4,LOC100758113,{100758113}
...,...,...
6065,LOC113832931,{113832931}
6066,LOC113836422,{113836422}
6067,LOC100765745,{100765745}
6068,LOC113837460,{113837460}


In [9]:
genes_silenced[genes_silenced["geneID set"].apply(lambda x: len(x)> 1)]

,0,geneID set
5677,Centrin-1,"{100757090, 100768500}"


## One gene has two gene IDs. We can just take either with pop

In [10]:
genes_silenced["geneID"] = genes_silenced["geneID set"].apply(lambda x: list(x)[0])
genes_silenced[['geneID']].to_csv(join(outdir, "geneID_silenced.csv"), index=False)

In [11]:
len(genes_silenced["geneID set"])

6070